In [1]:
from EGraphSAGE import Model, train
import pickle
import dgl
import networkx
import torch as th

/home/joel/working/NIDS-HPO/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
with open("./interm/proto_graph.pkl", "rb") as f:
    G = pickle.load(f)
    print('loaded prototype graph')

loaded prototype graph


In [3]:
type(G)

networkx.classes.multigraph.MultiGraph

In [4]:
if type(G) == networkx.classes.multigraph.MultiGraph:
    G = G.to_directed()
    G = dgl.from_networkx(G,edge_attrs=['h','Attack'] )
    G.ndata['h'] = th.ones(G.num_nodes(), G.edata['h'].shape[1])
    G.ndata['h'] = th.reshape(G.ndata['h'], (G.ndata['h'].shape[0], 1,G.ndata['h'].shape[1]))
    G.edata['h'] = th.reshape(G.edata['h'], (G.edata['h'].shape[0], 1,G.edata['h'].shape[1]))

In [5]:
import torch.nn.functional as F

model = Model(
    ndim_in=G.ndata['h'].shape[2], 
    ndim_out=128, 
    edim=G.ndata['h'].shape[2], 
    activation=F.relu, 
    dropout=0.2)#.cuda()

print(model)

Model(
  (gnn): SAGE(
    (layers): ModuleList(
      (0): SAGELayer(
        (W_msg): Linear(in_features=100, out_features=128, bias=True)
        (W_apply): Linear(in_features=178, out_features=128, bias=True)
      )
      (1): SAGELayer(
        (W_msg): Linear(in_features=178, out_features=128, bias=True)
        (W_apply): Linear(in_features=256, out_features=128, bias=True)
      )
    )
    (dropout): Dropout(p=0.2, inplace=False)
  )
  (pred): MLPPredictor(
    (W): Linear(in_features=256, out_features=10, bias=True)
  )
)


In [6]:
len(G.edata['h'])

236544

In [7]:
import numpy as np

l = len(G.edata['h'])
tr = int(l * 0.8)
o = l - tr
edge_train_mask = np.concatenate((np.ones(tr), np.zeros(o)))
valid = int(o*0.5)
edge_valid_mask = np.concatenate((np.zeros(tr), np.ones(valid), np.zeros(o - valid)))

In [8]:
len(edge_train_mask)

236544

In [10]:
from sklearn.utils import class_weight  
from torch import nn

class_weights = class_weight.compute_class_weight(
    class_weight='balanced',
     classes=np.unique(G.edata['Attack'].numpy()),
     y=G.edata['Attack'].numpy())

class_weights = th.FloatTensor(class_weights)
criterion = nn.CrossEntropyLoss(weight = class_weights)

In [13]:
import dgl.function as fn
from EGraphSAGE import compute_accuracy
from tqdm import tqdm

epochs = 100

node_features = G.ndata['h']
edge_features = G.edata['h']
edge_label = G.edata['Attack']
# train_mask = G.edata['train_mask']

opt = th.optim.Adam(model.parameters())

for epoch in tqdm(range(1, epochs)):
    pred = model(G, node_features, edge_features)

    loss = criterion(pred[edge_train_mask], edge_label[edge_train_mask])
    opt.zero_grad()
    loss.backward()
    opt.step()
    if epoch % 5 == 0:
        print('Training acc:', compute_accuracy(pred[edge_train_mask], 
                                                edge_label[edge_train_mask]))
        print('Validation acc:', compute_accuracy(pred[edge_valid_mask], 
                                                  edge_label[edge_valid_mask]))

    if epoch == epochs-1:
        test_mask = ~np.array(edge_train_mask + edge_valid_mask, dtype=bool)
        print('\nFinal test acc:', compute_accuracy(pred[test_mask], 
                                                    edge_label[test_mask]))


  2%|▏         | 2/99 [00:23<18:41, 11.56s/it]


KeyboardInterrupt: 